<a target="_blank" href="https://colab.research.google.com/github/segmed/gemma_mistral/blob/main/notebooks/datasets_example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
!pip install -qq datasets pandas

In [5]:
from datasets import load_dataset
dataset = load_dataset('medmcqa', split='train').shuffle(seed=42)
dataset

Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6150 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4183 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 182822
})

In [7]:
import pandas as pd
df = pd.DataFrame(dataset)
df

,id,question,opa,opb,opc,opd,cop,choice_type,exp,subject_name,topic_name
0,c69ce981-e254-4707-b87a-cb493e8a948e,OPV can be used if vaccine l monitor is showing?,Colour of outer circle is same as inner square,Colour of outer circle is darker than inner sq...,Colour of outer circle is lighter than inner s...,None of the above,1,multi,"Ans. is 'b' i.e., Colour of outer circle is da...",Social & Preventive Medicine,None
1,4bf5d147-3860-4dcf-ac71-9f9b0a87e583,Average gain of height in first year ?,25 cms,50 cms,75 cms,100 cms,0,single,O.P. Ghai 9th Edition - page 13,Anatomy,General anatomy
2,a524efbb-8a52-4298-9f4f-40790d95c8b9,Investigation of choice to diagnose congenital...,USG,HSG,MRI,CT,2,single,MRI is the investigation of choice to diagnose...,Gynaecology & Obstetrics,None
3,8d5a6669-3645-4275-a1da-89ee456bf3fd,Pain is appreciated when the small bowel is,Burned,crushed,Cut,distended,3,multi,The gut is insensitive to common painful stimu...,Surgery,None
4,e778cda1-38c5-4583-bf9d-02979ff48c67,The main advantage of RPD over FPD in replacin...,Cross arch stabilization,Aesthetics,Less expensive,Comfort to the patient,0,single,None,Dental,None
...,...,...,...,...,...,...,...,...,...,...,...
182817,e1a6cfaf-36cd-465e-8d96-6cd4916b5345,"When one strand of DNA isolated showed 20A, 25...","A=45 G=45, C=52, T=52","A=50, G=47, C=50, T=47","A=44, G=55, C=55, T=44","A=42, G=55, C=55, T=42",3,multi,Chargaff's rule,Biochemistry,None
182818,8d7aefb7-53b2-4e8a-a39e-a80c292657de,Vasa vasora are functionally analogous to ?,Valves,Basal lamina,Coronary aeries,Endothelial diaphragms,2,multi,Vasa vasorum are small blood vessels that supp...,Anatomy,General anatomy
182819,654799d5-2a02-4908-94fe-1119512b6df9,All of the following are treatment modalities ...,Chemotherapy for primary gastric lymphoma,Chemotherapy for secondary gastric lymphoma,Antibiotic treatment for H. pylori in low grad...,Radical subtotal gastrectomy for locally advan...,3,multi,Ans. (d) Radical subtotal gastrectomy for loca...,Surgery,Stomach & Duodenum
182820,e086b206-96fe-45a6-8698-209fcbba16a1,All of the following are disorders of phagocyt...,Chronic granulomatous disease,X-linked SCID,Chediak-Higashi syndrome,Myeloperoxidase deficiency,1,multi,X-linked Severe Combined Immunodeficiency (SCI...,Pathology,Oxygen dependent bacterial killing


In [9]:
df['subject_name'].value_counts()

Medicine                        17887
Surgery                         16862
Pathology                       14884
Anatomy                         14560
Pharmacology                    13758
Social & Preventive Medicine    11882
Microbiology                    11314
Gynaecology & Obstetrics        10013
Dental                           8938
Physiology                       8830
Biochemistry                     8282
Pediatrics                       8037
Ophthalmology                    6932
Forensic Medicine                5900
ENT                              4919
Psychiatry                       4442
Radiology                        4395
Anaesthesia                      3172
Unknown                          3045
Orthopaedics                     2999
Skin                             1771
Name: subject_name, dtype: int64

In [10]:
 df['question'].apply(len).mean()

78.70989268249991